In [1]:
from utils import *
from model import *
import sys
import torch.optim as optim
from torch import nn
import torch

In [7]:
train_file = '../data/train-pos-neg-classification.csv'
test_file = '../data/test-pos-neg-classification.csv'
val_file = '../data/val-pos-neg-classification.csv'
w2v_file = '../data/ubercorpus.cased.tokenized.word2vec.300d'

In [13]:
class Config(object):
    embed_size = 300
    hidden_layers = 1
    hidden_size = 64
    output_size = 4
    max_epochs = 15
    hidden_size_linear = 64
    lr = 0.5
    batch_size = 128
    seq_len = None # Sequence length for RNN
    dropout_keep = 0.8


In [9]:
config = Config()
dataset = Dataset(config)
dataset.load_data(w2v_file, train_file, test_file, val_file)

Loaded 51641 training examples
Loaded 19211 test examples
Loaded 16369 validation examples


#### Create Model with specified optimizer and loss function

In [10]:
model = RCNN(config, len(dataset.vocab), dataset.word_embeddings)
if torch.cuda.is_available():
    model.cuda()
model.train()
optimizer = optim.SGD(model.parameters(), lr=config.lr)
NLLLoss = nn.NLLLoss()
model.add_optimizer(optimizer)
model.add_loss_op(NLLLoss)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [11]:
train_losses = []
val_accuracies = []

for i in range(config.max_epochs):
    print ("Epoch: {}".format(i))
    train_loss,val_accuracy = model.run_epoch(dataset.train_iterator, dataset.val_iterator, i)
    train_losses.append(train_loss)
    val_accuracies.append(val_accuracy)

Epoch: 0


/Volumes/data/Data Science/Diploma/external_projects/Text-Classification-Models-Pytorch/Model_RCNN/model.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(final_out)


Iter: 1
	Average training loss: -0.22072


KeyboardInterrupt: 

In [12]:
train_acc = evaluate_model(model, dataset.train_iterator)
val_acc = evaluate_model(model, dataset.val_iterator)
test_acc = evaluate_model(model, dataset.test_iterator)

print ('Final Training Accuracy: {:.4f}'.format(train_acc))
print ('Final Validation Accuracy: {:.4f}'.format(val_acc))
print ('Final Test Accuracy: {:.4f}'.format(test_acc))

KeyboardInterrupt: 